In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Machine Learning: Text Classification Assignment

In [2]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [5]:
PATH = '/content/drive/MyDrive/python_for_data_scientists/AP_News'

DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+)/.*'

corpus = CategorizedPlaintextCorpusReader(PATH, DOC_PATTERN, cat_pattern=CAT_PATTERN)

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [6]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]
categories = [corpus.categories(fileid)[0] for fileid in corpus.fileids()]

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [8]:
def preprocess(docs):
  lemmatizer = WordNetLemmatizer()
  stemmer = SnowballStemmer('english')
  preprocessed = []

  for doc in docs:
    tokenized = word_tokenize(doc)
    cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower())) 
               for token in tokenized 
               if not token.lower() in stopwords.words('english') 
               if token.isalpha()]

    untokenized = ' '.join(cleaned)
    preprocessed.append(untokenized)
  return preprocessed

In [15]:
good_to_go = preprocess(docs)

In [ ]:
good_to_go

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [17]:
x_train, x_test, y_train, y_test = tts(good_to_go, categories, test_size=0.3)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [25]:
pline = Pipeline([
                  ('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators=100))
])

pline.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [26]:
preds = pline.predict(x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

      health       0.85      0.92      0.88        12
    politics       0.83      0.91      0.87        22
      sports       0.86      0.80      0.83        15
        tech       0.87      0.76      0.81        17

    accuracy                           0.85        66
   macro avg       0.85      0.85      0.85        66
weighted avg       0.85      0.85      0.85        66



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [27]:
scores = cross_val_score(pline, good_to_go, categories, cv=10, scoring='f1_macro')
scores.mean()

0.7828500502765209

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [21]:
import requests
from bs4 import BeautifulSoup

In [30]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'
url_ = 'https://www.npr.org/2020/12/29/951010863/u-s-announces-new-rules-for-drones-and-their-operators'

In [28]:
def get_text(url):
  response = requests.get(url)
  content = response.text

  TAGS = [ 'h1','h2','h3','h4','h5','h6','h7','p','li']
  soup = BeautifulSoup(content, 'lxml')
  text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
  text = ' '.join(text_list)
  return text

In [31]:
text = get_text(url_)
cleaned = preprocess([text])
pline.predict(cleaned)[0]

'health'

Accurate predicting the first article but not the second. 